In [62]:
% % capture
import sys

if 'google.colab' in sys.modules:
    # Install packages in Colab
    !pip install category_encoders == 2. *

# If you're working locally:
else:
    DATA_PATH = '../data/'

In [63]:
#import our libraries here
% matplotlib inline
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

import category_encoders as ce
from sklearn.impute import SimpleImputer
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.pipeline import make_pipeline
from sklearn.ensemble import RandomForestClassifier
from sklearn.feature_selection import f_regression, SelectKBest
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import plot_confusion_matrix
from sklearn.metrics import r2_score, mean_squared_error

In [64]:
!wget https: // raw.githubusercontent.com / DS-buildweek-ttdspt178 / front-end / new-features / Data / AB_US_2020.csv

--2021-04-26 00:31:34--  https://raw.githubusercontent.com/DS-buildweek-ttdspt178/front-end/new-features/Data/AB_US_2020.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.111.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 35857419 (34M) [text/plain]
Saving to: ‘AB_US_2020.csv.3’

AB_US_2020.csv.3    100%[===================>]  34.20M  8.81MB/s    in 4.9s    

2021-04-26 00:31:40 (7.05 MB/s) - ‘AB_US_2020.csv.3’ saved [35857419/35857419]



In [65]:
df = pd.read_csv('AB_US_2020.csv')

/home/rob-rog/deps/anaconda3/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3146: DtypeWarning: Columns (4) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


In [66]:
#lets see the first 5 rows
df.head()

,id,name,host_id,host_name,neighbourhood_group,neighbourhood,latitude,longitude,room_type,price,minimum_nights,number_of_reviews,last_review,reviews_per_month,calculated_host_listings_count,availability_365,city
0,38585,Charming Victorian home - twin beds + breakfast,165529,Evelyne,NaN,28804,35.65146,-82.62792,Private room,60,1,138,16/02/20,1.14,1,0,Asheville
1,80905,French Chic Loft,427027,Celeste,NaN,28801,35.59779,-82.55540,Entire home/apt,470,1,114,07/09/20,1.03,11,288,Asheville
2,108061,Walk to stores/parks/downtown. Fenced yard/Pet...,320564,Lisa,NaN,28801,35.60670,-82.55563,Entire home/apt,75,30,89,30/11/19,0.81,2,298,Asheville
3,155305,Cottage! BonPaul + Sharky's Hostel,746673,BonPaul,NaN,28806,35.57864,-82.59578,Entire home/apt,90,1,267,22/09/20,2.39,5,0,Asheville
4,160594,Historic Grove Park,769252,Elizabeth,NaN,28801,35.61442,-82.54127,Private room,125,30,58,19/10/15,0.52,1,0,Asheville


In [67]:
#check for missing values
df.isnull().sum()

id                                     0
name                                  28
host_id                                0
host_name                             33
neighbourhood_group               115845
neighbourhood                          0
latitude                               0
longitude                              0
room_type                              0
price                                  0
minimum_nights                         0
number_of_reviews                      0
last_review                        48602
reviews_per_month                  48602
calculated_host_listings_count         0
availability_365                       0
city                                   0
dtype: int64

In [68]:
#remove NaN values from dataframe
original = len(df)
df = df.dropna(how='any', subset=['name', 'host_name', 'neighbourhood_group',
                                  'last_review', 'reviews_per_month'])
# print 'Number of NaN values removed:',original - len(df)

In [69]:
df.dtypes

id                                  int64
name                               object
host_id                             int64
host_name                          object
neighbourhood_group                object
neighbourhood                      object
latitude                          float64
longitude                         float64
room_type                          object
price                               int64
minimum_nights                      int64
number_of_reviews                   int64
last_review                        object
reviews_per_month                 float64
calculated_host_listings_count      int64
availability_365                    int64
city                               object
dtype: object

In [70]:
#check for null values
df.isnull().sum()

id                                0
name                              0
host_id                           0
host_name                         0
neighbourhood_group               0
neighbourhood                     0
latitude                          0
longitude                         0
room_type                         0
price                             0
minimum_nights                    0
number_of_reviews                 0
last_review                       0
reviews_per_month                 0
calculated_host_listings_count    0
availability_365                  0
city                              0
dtype: int64

In [71]:
#check for the last 5 rows
df.tail()

,id,name,host_id,host_name,neighbourhood_group,neighbourhood,latitude,longitude,room_type,price,minimum_nights,number_of_reviews,last_review,reviews_per_month,calculated_host_listings_count,availability_365,city
212157,43531859,Chic Seattle Apartment near Greenlake,6601753,Isabel,Other neighborhoods,Roosevelt,47.68457,-122.31550,Entire home/apt,100,3,1,06/06/20,1.0,1,99,Seattle
212159,43554835,SL6 - Private¬∑Modern¬∑Quality¬∑Convenient¬∑N ...,287025852,Li,Lake City,Olympic Hills,47.73462,-122.29509,Private room,79,1,2,10/06/20,2.0,3,337,Seattle
212160,43554849,"SL4 - Modern,Private,Quality,Convenient,N Seattle",287025852,Li,Lake City,Olympic Hills,47.73357,-122.29651,Entire home/apt,79,1,1,13/06/20,1.0,3,351,Seattle
212161,43589616,Perfect spot for group events or family stays!,347974040,Marcus,Delridge,Highland Park,47.51284,-122.33587,Entire home/apt,200,1,2,14/06/20,2.0,1,330,Seattle
212164,43603849,Rooftop Tent in the Heart of Seattle,51816582,Fei,Queen Anne,East Queen Anne,47.63913,-122.34293,Private room,35,1,2,14/06/20,2.0,2,0,Seattle


###Train test Split here

In [72]:
#lets do the train test split here
train, test = train_test_split(df, train_size=0.80, test_size=0.20,
                               random_state=42)
train.shape, test.shape

((68115, 17), (17029, 17))

In [73]:
target = 'price'
guess = df[target].mean()
print('{:,.2f}'.format(guess))

173.86


In [74]:
#splitting the data
X_train = train.drop(columns=[target, 'name', 'host_name', 'neighbourhood_group',
                              'neighbourhood', 'room_type', 'city'])
y_train = train[target]
X_test = test.drop(columns=[target, 'name', 'host_name', 'neighbourhood_group', 'neighbourhood', 'room_type', 'city'])
y_test = test[target]
X_train.shape, y_train.shape, X_test.shape, y_test.shape

((68115, 10), (68115,), (17029, 10), (17029,))

In [75]:
#arrange our data here
X_train = train.drop(columns=[target, 'name', 'host_name', 'neighbourhood_group', 'neighbourhood',
                              'room_type', 'city'])
y_train = train[target]
X_test = test.drop(columns=[target, 'name', 'host_name', 'neighbourhood_group', 'neighbourhood', 'room_type', 'city'])
y_test = test[target]

#check the shape 
X_train.shape, y_train.shape, X_test.shape, y_test.shape

((68115, 10), (68115,), (17029, 10), (17029,))

In [76]:

def __init__(self, model):
    self.model = model
    self.x_train = None
    self.y_train = None
    self.x_test = None
    self.y_test = None
    self.y_pred_train = None
    self.y_pred_test = None
    self.train_score = None
    self.test_score = None
    self.train_score_log = None
    self.test_score_log = None

    def data_split(self, x, y, test_size):
        self.x_train, self.x_test, self.y_train, self.y_test = train_test_split(x, y, test_size=test_size)


def score_reg(self):
    return self.train_score, self.test_score


def score_log(self):
    self.train_score_log = metrics.r2_score(np.exp(self.y_train), np.exp(self.y_pred_train))
    self.test_score_log = metrics.r2_score(np.exp(self.y_test), np.exp(self.y_pred_test))
    return self.train_score_log, self.test_score_log


def data_frame_convert(self):
    df_train = pd.DataFrame({'y_pred': self.y_pred_train, 'y_real': self.y_train})
    df_test = pd.DataFrame({'y_pred_test': self.y_pred_test, 'y_real_test': self.y_test})
    return self.train_score, self.test_score, df_train, df_test


def data_frame_convert_log(self):
    df_train = pd.DataFrame({'y_pred': np.exp(self.y_pred_train), 'y_real': np.exp(self.y_train)})
    df_test = pd.DataFrame({'y_pred_test': np.exp(self.y_pred_test), 'y_real_test': np.exp(self.y_test)})
    return self.train_score_log, self.test_score_log, df_train, df_test


def fit_model(self, x, y, test_size):
    self.data_split(x, y, test_size)
    self.model = self.model.fit(self.x_train, self.y_train)
    self.train_score = self.model.score(self.x_train, self.y_train)
    self.test_score = self.model.score(self.x_test, self.y_test)
    self.y_pred_train = self.model.predict(self.x_train)
    self.y_pred_test = self.model.predict(self.x_test)

# def model_iterations(n, x, y, model_arg, log_bool=False):
#   training_scores = [None]*n
#   testing_scores = [None]*n

# for i in range(n):
#     new_model = model(model_arg)
#     new_model.fit_model(x, y, 0.3)
#     training_scores[i], testing_scores[i] = new_model.score_reg() if not log_bool else new_model.score_log()

# print ('Mean Train Score:', np.mean(training_scores))
# print ('Mean Test Score:', np.mean(testing_scores))
# return new_model

###Convert Categorical Variables to dummy integer values here

In [77]:
#arranging the y target vectors
target = 'price'
y_train = df[target]

In [78]:
#Lets make first baseline guess
print('Mean baseline(using 0 features)')
baseline = y_train.mean()
print(baseline)

Mean baseline(using 0 features)
173.85819317861504


In [79]:
#Lets make first baseline guess
print('Mean baseline(using 0 features)')
baseline = y_train.mean()
print(baseline)

Mean baseline(using 0 features)
173.85819317861504


In [80]:
from sklearn.metrics import mean_absolute_error, r2_score

y_pred = [baseline] * len(y_train)
mae = mean_absolute_error(y_train, y_pred)
r2 = r2_score(y_train, y_pred)
print(mae)
print(r2)

114.14646497940572
0.0


###Simple Regression Model

In [81]:
#OLS regression
# model = LinearRegression(copy_X=True, fit_intercept=True, n_jobs=1, normalize=False)
# model.fit(x_train, y_train)
# predicted = clf.predict(x_test)

# score = sk.metrics.r2_score(y_test, predicted)
# print('sklearn: R2 score for Linear Regression is: {}'.format(score))

---

In [82]:
#arrange our data here
# X_train = train.drop(columns=[target, 'name','host_name', 'neighbourhood', 'neighbourhood_group', 'last_review'])
X_train = train.drop(columns=[target, 'id', 'host_id', 'name', 'host_name', 'neighbourhood', 'last_review'])
y_train = train[target]
X_test = test.drop(columns=[target, 'id', 'host_id', 'name', 'host_name', 'neighbourhood', 'last_review'])
y_test = test[target]

#check the shape 
X_train.shape, y_train.shape, X_test.shape, y_test.shape

((68115, 10), (68115,), (17029, 10), (17029,))

In [83]:
train.describe(include='all')

,id,name,host_id,host_name,neighbourhood_group,neighbourhood,latitude,longitude,room_type,price,minimum_nights,number_of_reviews,last_review,reviews_per_month,calculated_host_listings_count,availability_365,city
count,6.811500e+04,68115,6.811500e+04,68115,68115,68115,68115.000000,68115.000000,68115,68115.000000,68115.000000,68115.000000,68115,68115.000000,68115.000000,68115.000000,68115
unique,NaN,67096,NaN,14098,34,624,NaN,NaN,4,NaN,NaN,NaN,2213,NaN,NaN,NaN,5
top,NaN,Home away from home,NaN,Michael,Manhattan,Primary Urban Center,NaN,NaN,Entire home/apt,NaN,NaN,NaN,07/09/20,NaN,NaN,NaN,New York City
freq,NaN,21,NaN,542,12006,2547,NaN,NaN,44981,NaN,NaN,NaN,797,NaN,NaN,NaN,28078
mean,2.240598e+07,NaN,8.194389e+07,NaN,NaN,NaN,35.374106,-106.259206,NaN,174.439521,10.123996,37.759554,NaN,1.154306,12.256419,157.939632,NaN
std,1.275165e+07,NaN,9.026877e+07,NaN,NaN,NaN,8.091471,32.338521,NaN,309.145709,24.162486,61.892276,NaN,1.463953,36.338795,141.279766,NaN
min,1.090000e+02,NaN,5.210000e+02,NaN,NaN,NaN,18.920990,-159.714900,NaN,0.000000,1.000000,1.000000,NaN,0.010000,1.000000,0.000000,NaN
25%,1.222313e+07,NaN,1.073794e+07,NaN,NaN,NaN,33.978665,-122.306880,NaN,75.000000,2.000000,3.000000,NaN,0.170000,1.000000,0.000000,NaN
50%,2.207866e+07,NaN,4.331589e+07,NaN,NaN,NaN,40.638680,-118.170480,NaN,119.000000,3.000000,12.000000,NaN,0.560000,1.000000,142.000000,NaN
75%,3.345157e+07,NaN,1.314708e+08,NaN,NaN,NaN,40.746205,-73.955715,NaN,190.000000,7.000000,44.000000,NaN,1.630000,4.000000,309.000000,NaN


In [84]:
X_train.head()

,neighbourhood_group,latitude,longitude,room_type,minimum_nights,number_of_reviews,reviews_per_month,calculated_host_listings_count,availability_365,city
101437,Other Cities,34.12804,-118.24042,Entire home/apt,1,17,4.95,1,361,Los Angeles
134849,Brooklyn,40.57577,-73.98553,Entire home/apt,1,8,0.30,6,364,New York City
75003,Other Cities,34.04065,-118.48329,Private room,2,252,3.59,1,360,Los Angeles
130659,Manhattan,40.76026,-73.96590,Private room,1,329,6.43,1,0,New York City
89291,City of Los Angeles,34.14262,-118.40379,Entire home/apt,30,22,0.93,1,26,Los Angeles


In [99]:
# this is a regression problem

from sklearn.linear_model import LinearRegression

import category_encoders as ce
from sklearn.ensemble import RandomForestRegressor
from sklearn.pipeline import make_pipeline
from sklearn.impute import SimpleImputer

# instantiate a pipeline with Simple Imputer and Random Forest Regressor
pipeline = make_pipeline(
    ce.OneHotEncoder(use_cat_names=True),
    LinearRegression()
)

# fit on train 
pipeline.fit(X_train, y_train)

# print the Test Score
print('Test Score:', pipeline.score(X_test, y_test))

Test Score: 0.11535529582055715


/home/rob-rog/deps/anaconda3/lib/python3.8/site-packages/category_encoders/utils.py:21: FutureWarning: is_categorical is deprecated and will be removed in a future version.  Use is_categorical_dtype instead
  elif pd.api.types.is_categorical(cols):


In [86]:
X_train.head(1)

,neighbourhood_group,latitude,longitude,room_type,minimum_nights,number_of_reviews,reviews_per_month,calculated_host_listings_count,availability_365,city
101437,Other Cities,34.12804,-118.24042,Entire home/apt,1,17,4.95,1,361,Los Angeles


In [87]:
X_train.columns

Index(['neighbourhood_group', 'latitude', 'longitude', 'room_type',
       'minimum_nights', 'number_of_reviews', 'reviews_per_month',
       'calculated_host_listings_count', 'availability_365', 'city'],
      dtype='object')

In [88]:
y_train.head(1)

101437    124
Name: price, dtype: int64

In [89]:
# making a prediction and seeing the difference from actual on a test data

sample_data = {'neighbourhood_group': 'Other Cities', 'latitude': 34.12804, 'longitude': -118.24042,
               'room_type': 'Entire home/apt', 'minimum_nights': 1, 'number_of_reviews': 17, 'reviews_per_month': 4.95,
               'calculated_host_listings_count': 1, 'availability_365': 361, 'city': 'Los Angeles'}

sample_data

# change the dictionary to dataframe
sample_data_df = pd.DataFrame([sample_data])

y_pred = pipeline.predict(sample_data_df)
y_pred

array([150.96])

In [90]:
sample_data_df

,neighbourhood_group,latitude,longitude,room_type,minimum_nights,number_of_reviews,reviews_per_month,calculated_host_listings_count,availability_365,city
0,Other Cities,34.12804,-118.24042,Entire home/apt,1,17,4.95,1,361,Los Angeles


In [100]:
from joblib import dump

dump(pipeline, 'pipeline.joblib', compress=True)

['pipeline.joblib']

In [92]:
!ls -lh

total 233M
-rw-rw-r-- 1 rob-rog rob-rog  35M Apr 25 23:09 AB_US_2020.csv
-rw-rw-r-- 1 rob-rog rob-rog  35M Apr 25 23:30 AB_US_2020.csv.1
-rw-rw-r-- 1 rob-rog rob-rog  35M Apr 25 23:44 AB_US_2020.csv.2
-rw-rw-r-- 1 rob-rog rob-rog  35M Apr 26 00:31 AB_US_2020.csv.3
-rw-rw-r-- 1 rob-rog rob-rog 337K Apr 24 22:37 Model_deployment_with_FastAPI.ipynb
-rw-rw-r-- 1 rob-rog rob-rog  96M Apr 26 00:31 pipeline.joblib
-rw-rw-r-- 1 rob-rog rob-rog  47K Apr 26 00:31 Unit3ProjectAbdiWork.ipynb


In [95]:
X_train.describe(include='all')

,neighbourhood_group,latitude,longitude,room_type,minimum_nights,number_of_reviews,reviews_per_month,calculated_host_listings_count,availability_365,city
count,68115,68115.000000,68115.000000,68115,68115.000000,68115.000000,68115.000000,68115.000000,68115.000000,68115
unique,34,NaN,NaN,4,NaN,NaN,NaN,NaN,NaN,5
top,Manhattan,NaN,NaN,Entire home/apt,NaN,NaN,NaN,NaN,NaN,New York City
freq,12006,NaN,NaN,44981,NaN,NaN,NaN,NaN,NaN,28078
mean,NaN,35.374106,-106.259206,NaN,10.123996,37.759554,1.154306,12.256419,157.939632,NaN
std,NaN,8.091471,32.338521,NaN,24.162486,61.892276,1.463953,36.338795,141.279766,NaN
min,NaN,18.920990,-159.714900,NaN,1.000000,1.000000,0.010000,1.000000,0.000000,NaN
25%,NaN,33.978665,-122.306880,NaN,2.000000,3.000000,0.170000,1.000000,0.000000,NaN
50%,NaN,40.638680,-118.170480,NaN,3.000000,12.000000,0.560000,1.000000,142.000000,NaN
75%,NaN,40.746205,-73.955715,NaN,7.000000,44.000000,1.630000,4.000000,309.000000,NaN


In [94]:
X_test.columns

Index(['neighbourhood_group', 'latitude', 'longitude', 'room_type',
       'minimum_nights', 'number_of_reviews', 'reviews_per_month',
       'calculated_host_listings_count', 'availability_365', 'city'],
      dtype='object')

In [93]:
# making sure the data is passed in a certain way

from pydantic import BaseModel, confloat, constr, conint


class Airbnb(BaseModel):
    neighbourhood_group: constr(),
    latitude: confloat(gt=18, le=48),
    longitude: confloat(gt=-106, le=-72),
    room_type: constr(),
    minimum_nights: conint(gt=1, le=1250),
    number_of_reviews: conint(gt=1, le=840),
    reviews_per_month: confloat(gt=0.01, le=34),
    calculated_host_listings_count: conint(gt=1, le=284),
    availability_365: conint(gt=0, le=365),
    city: constr()

    def to_df(self):
        return pd.DataFrame([dict(self)])



In [ ]:
{
    "neighbourhood_group": "Other Cities",
    "latitude": 34.12804,
    "longitude": -118.24042,
    "room_type": "Entire home/apt",
    "minimum_nights": 1,
    "number_of_reviews": 17,
    "reviews_per_month": 4.95,
    "calculated_host_listings_count": 1,
    "availability_365": 361,
    "city": "Los Angeles"
}


{
  "neighbourhood_group": "Other Cities",
  "latitude": 34.12804,
  "longitude": -118.2402,
  "room_type": "Entire home/apt",
  "minimum_nights": 1,
  "number_of_reviews": 17,
  "reviews_per_month": 4.95,
  "calculated_host_listings_count": 1,
  "availability_365": 361,
  "city": "Los Angeles"
}